In [0]:
from google.colab import files
from google.colab import drive
import os
import pandas as pd
import pickle
import os
import time
from statistics import mean
import math
from PIL import Image
import numpy as np
import pandas as pd
from IPython.display import display
import pickle as pkl
import multiprocessing
import threading
from heapq import nlargest
import time

In [30]:
drive.mount('/content/drive/')
os.chdir('drive/My Drive/Winter 2020/MCA Assignments/Assignment-1/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


FileNotFoundError: ignored

In [31]:
os.getcwd()

'/content/drive/My Drive/Winter 2020/MCA Assignments/Assignment-1'

Data Preprocessing

In [0]:
def resize(img, factor):
    return img.resize((int(img.size[0]/factor), int(img.size[1]/factor)))

def quantize(img, n_colors):
    return img.quantize(n_colors)

def process(img_src):
    img = Image.open(img_src)
    img = quantize(img, 64)
    img = resize(img, 4)
    width, height = img.size
    processed = np.array(list(img.getdata())).reshape((width, height))
    return processed

In [0]:
directory = 'images/'
image_pixels = {}
print('Preprocessing images and converting into pixels')
i=1
for f in os.listdir(directory):
    # print(f)
    if f.endswith(".jpg"):
        print(str(i)+ ' '+ f)
        image_pixels[f] = process(directory + f)
        i+=1

In [0]:
pkl.dump(image_pixels, open('image_pixels.pkl', 'wb'))

**Color Auto-Correlogram** \\
credits: [Mathworks: Color Auto-Correlogram](https://in.mathworks.com/matlabcentral/fileexchange/46093-color-auto-correlogram) 

In [0]:
# Helper functions    

def get_nbrh(d, x, y, color, I, X, Y):    
    positive_count = 0   
    total_count = 0
    #property of inf-norm, every distance is 8 times the order.
    x_nbrs = np.zeros(8*d)  
    y_nbrs = np.zeros(8*d)

    #X-coordinates
    #from 0 to d
    x_nbrs.put(list(range(d)), x-d)

    #from d to 3d  
    n = 0
    for k in range(d,3*d):
        x_nbrs[k] = x - d + n
        n+=1

    #from 3d to 5d
    x_nbrs.put(list(range(3*d, 5*d)), x+d)

    #from 5d to 7d
    n = 0
    for k in range(5*d,7*d):
        x_nbrs[k] = x + d - n
        n+=1
    
    #from 7d to end
    x_nbrs.put(list(range(7*d, 8*d)), x-d)    


    #Y-coordinates 
    #from 1 to d
    y_nbrs[0] = y
    n = 1 
    for k in range(1,d):
        y_nbrs[k] = y - n
        n+=1

    #from d to 3d
    y_nbrs.put(list(range(d,3*d)), y-d)
        
    #from 3d to 5d
    n = 0
    for k in range(3*d,5*d):
        y_nbrs[k] = y - d + n
        n+=1
        
    #from 5d to 7d
    y_nbrs.put(list(range(5*d,7*d)), y+d)

    #from 7d to 8d
    n = 0
    for k in range(7*d,8*d):
        y_nbrs[k] = y + d - n
        n+=1   

    # Counting common number of colors in the neighborhood
    for i in range(8*d):
        if x_nbrs[i] >= 0 and x_nbrs[i] < X:
            if y_nbrs[i] >= 0 and y_nbrs[i] < Y:
                nbr_color = I[int(y_nbrs[i])][int(x_nbrs[i])]
                if nbr_color == color or abs(nbr_color-color)==1:
                    positive_count += 1
                total_count += 1

    return positive_count, total_count

In [0]:
m = 64

def color_auto_corr(I, dist_vector):
    correlogram_vector = []
    d = len(dist_vector)
    count = np.zeros((m, d))
    total = np.zeros((m, d))
    prob = np.zeros((d, m))
    height, width = np.shape(I)
    
    for i in range(d):
        for x in range(width):
            for y in range(height):
                color = I[y][x]
                positive_count, total_count = get_nbrh(dist_vector[i], x, y, color, I, width, height)
                count[color][i] = count[color][i] + positive_count
                total[color][i] = total[color][i] + total_count 
        prob[i] = count[:,i]/total[:,i]

    for i in range(d):
        correlogram_vector.append(prob[i])

    return correlogram_vector

Calculate Features

In [0]:
image_pixels = pkl.load(open('image_pixels.pkl', 'rb'))

In [0]:
print(image_pixels)

In [0]:
image_features = {}
distances = [1,3,5]

images = sorted(image_pixels.keys())
n_threads = 10
reponsibility = []
start = 0
for i in range(n_threads):
    start = i*500
    stop = start + 500
    reponsibility.append(images[start:stop]) 
start += 500
extra = images[start:]
# print(extra)

k=1
for i in range(1):
    workers = []
    for j in range(n_threads):
        p = threading.Thread(target= color_auto_corr, args= (image_pixels[reponsibility[j][i]], distances)).
        # p = multiprocessing.Process(target= color_auto_corr, args= (image_pixels[reponsibility[j][i]], distances))
        workers.append(p)
        image_features[reponsibility[j][i]] = p
        workers[j].start()

    for j in range(n_threads):
        workers[j].join()
        # print('worker' + str(j) + ' resp' + str(i))
        print(k)
        k+=1

Image Retrieval

In [0]:
image_features = pkl.load(open('image_features.pkl', 'rb')) # Flattened Feature List

Similarity Measure

In [0]:
def similarity(query_corr, sample_corr, m):
    sum_arr = []
    for a,b in zip(query_corr, sample_corr):
        sum_arr.append(abs(a-b)/(1+a+b))
    return (1/m)*(sum(sum_arr))

def percent(num, denom):
    return num/denom * 100

In [0]:
total_imgs = 5063

directory = 'train/query/'
query = []

good_per = []
junk_per = []
ok_per = []

precision = []
recall = []
f1 = []

retrieval_time = []

for f in os.listdir(directory):
    print('Start Retrieval')
    start = time.time()  
    file = open('train/query/'+ f)
    line = file.readline()
    file.close()
    file_name = f
    # Without Bounding Box
    image_name = line.split(" ")[0][5:]
    query.append(image_name+'.jpg')
    query_corr = image_features.loc[image_name+'.jpg'].values.tolist()
    scores = {}
    for img in image_features.index:
        if (img != str(image_name+'.jpg')):
            scores[img] = similarity(query_corr, image_features.loc[img].values.tolist(), m)

    good_images = [line.rstrip('\n') for line in open('train/ground_truth/'+file_name[:-9]+'good.txt')]
    junk_images = [line.rstrip('\n') for line in open('train/ground_truth/'+file_name[:-9]+'junk.txt')]
    ok_images = [line.rstrip('\n') for line in open('train/ground_truth/'+file_name[:-9]+'ok.txt')]
    
    n = len(good_images) + len(junk_images) + len(ok_images) #top n images to be retrieved
    
    top_n = nlargest(n, scores, key = scores.get)
    
    end = time.time()
    print('End Retrieval')
    
    # Predicted Values present in ground-truth
    n_good = 0
    n_junk = 0
    n_ok = 0
    
    for a in top_n:
        if a[:-4] in good_images:
            n_good+=1
        if a[:-4] in junk_images:
            n_junk+=1
        if a[:-4] in ok_images:
            n_ok+=1 

    # Calculate Percentages
    ok_per.append(percent(n_ok,len(ok_images)))
    junk_per.append(percent(n_junk,len(junk_images)))
    good_per.append(percent(n_good,len(good_images)))

    tp = 0
    fn = 0 
    fp = 0

    for a in top_n:
        if a[:-4] in good_images or a[:-4] in junk_images or a[:-4] in ok_images:
            tp+=1
        else:
            fp+=1
    fn = n - tp

    # Calculate Precision-Recall
    precn = (tp)/(tp + fp)                  
    recal = (tp)/(tp + fn)                
    try:
        f_sc = 2 * (precn * recal) / (precn + recal)
    except ZeroDivisionError:
        f_sc = 0

    print(f + " Precision: " + str(precn) +" Recall: "+ str(recal) + " F1: "+str(f_sc))

    precision.append(precn)    
    recall.append(recal)        
    f1.append(f_sc)
    
    retrieval_time.append(end-start)

In [0]:
print("Avg Precision: "+ str(np.mean(precision)))
print("Avg Recall: "+ str(np.mean(recall)))
print("Avg F1: "+ str(np.mean(f1)))
print("Avg good: "+ str(np.mean(good_per)))
print("Avg junk: "+ str(np.mean(junk_per)))
print("Avg ok: "+ str(np.mean(ok_per)))